In [1]:
from __future__ import  absolute_import
from __future__ import  division
from __future__ import  print_function

In [2]:
import tensorflow as tf

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
import os
import sys
import argparse

In [4]:
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
FLAGS = None

In [6]:
def train():
    #导入数据
    mnist = input_data.read_data_sets(train_dir=FLAGS.data_dir,fake_data=FLAGS.fake_data)
    #建立会话
    sess = tf.InteractiveSession()
    #placeholder
    with tf.name_scope("input"):
        x = tf.placeholder(dtype=tf.float32,shape=[None,784],name="x-input")
        y_ = tf.placeholder(dtype=tf.int64,shape=[None],name="y-input")
    #输入
    with tf.name_scope("input-reshape"):
        image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
        tf.summary.image('input', image_shaped_input, 10)
    #定义weight变量函数
    def weight_vriables(shape):
        initial = tf.truncated_normal(shape,stddev=0.1)
        return tf.Variable(initial)
    #定义bias变量函数
    def bias_variable(shape):
        initial = tf.constant(0.1,shape=shape)
        return tf.Variable(initial)
    #汇总variable数据
    def variable_summary(var):
        with tf.name_scope("summary"):
            mean = tf.reduce_mean(var)
            tf.summary.scalar('mean',mean)
            with tf.name_scope('stddev'):
                stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
            tf.summary.scalar('stddev',stddev)
            tf.summary.scalar('max',tf.reduce_max(var))
            tf.summary.scalar('min',tf.reduce_min(var))
            tf.summary.histogram('histogram',var)
    #层内操作
    def nn_layer(input_tensor,input_dim,output_dim,layer_name,act = tf.nn.relu):
        with tf.name_scope(layer_name):
            with tf.name_scope('weights'):
                weights = weight_vriables([input_dim,output_dim])
                variable_summary(weights)
            with tf.name_scope('biases'):
                biases = bias_variable([output_dim])
                variable_summary(biases)
            with tf.name_scope('Wx_plus_b'):
                preactivate = tf.matmul(input_tensor,weights) + biases
                tf.summary.histogram('pre_activation',preactivate)
            activations = act(preactivate,name='activations')
            tf.summary.histogram('activations',activations)
            return activations
    #隐藏层
    hidden1 = nn_layer(x,784,500,'layer1') 
    # dropout层
    with tf.name_scope('dropout'):
        keep_prob = tf.placeholder(tf.float32)
        tf.summary.scalar('dropout_keep_probality',keep_prob)
        dropped = tf.nn.dropout(hidden1,keep_prob)
    #输出层
    y = nn_layer(hidden1,500,10,'layer2',act =tf.identity)

    with tf.name_scope('cross_entropy'):
        cross_entropy = tf.losses.sparse_softmax_cross_entropy(labels=y_,logits=y)
    tf.summary.scalar('cross_entropy',cross_entropy)
    #训练
    with tf.name_scope('train'):
        train_op = tf.train.AdamOptimizer(FLAGS.learning_rate).minimize(cross_entropy)
    #准确率
    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.equal(tf.argmax(y,1),y_)
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    tf.summary.scalar('accuracy',accuracy)
    # Merge all the summaries and write them out to
    # /tmp/tensorflow/mnist/logs/mnist_with_summaries (by default)
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(FLAGS.log_dir + '/train',graph=sess.graph)
    test_writer = tf.summary.FileWriter(FLAGS.log_dir + '/test')

    #初始化所有变量
    tf.global_variables_initializer().run()

    #Train the model, and also write summaries.
    # Every 10th step, measure test-set accuracy, and write test summaries
    # All other steps, run train_step on training data, & add training summaries
    def feed_dict(train):
        """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
        #训练
        if train or FLAGS.fake_data:
            xs, ys = mnist.train.next_batch(100, fake_data=FLAGS.fake_data)
            k = FLAGS.dropout
        #验证
        else:
            xs, ys = mnist.test.images, mnist.test.labels
            k = 1.0
        return {x: xs, y_: ys, keep_prob: k}
    for i in range(FLAGS.max_steps):
            if i % 10 == 0:
                summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
                test_writer.add_summary(summary,i)
                print('Accuracy at step %s: %s' % (i, acc))
            else:# Record train set summaries, and train
                if i % 100 == 99:
                    #tf.RunOptions定义TensorFlow运行选项，设置trace_lever FULL_TRACE
                    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                    #tf.RunMetadata()定义TensorFlow运行元信息，
                    #记录训练运算时间和内存占用等信息
                    run_metadata = tf.RunMetadata()
                    summary,_ = sess.run([merged,train_op],
                                         feed_dict=feed_dict(True),
                                         options = run_options,
                                         run_metadata= run_metadata)
                    train_writer.add_run_metadata(run_metadata,'step%03d' % i)
                    train_writer.add_summary(summary,i)
                    print('Adding run metadata for', i)
                else:# Record a summary
                    summary,_ = sess.run([merged,train_op],feed_dict=feed_dict(True))
                    train_writer.add_summary(summary,i)
    train_writer.close()
    test_writer.close()

In [7]:
#主函数
def main(_):
    if tf.gfile.Exists(FLAGS.log_dir):
        tf.gfile.DeleteRecursively(FLAGS.log_dir)
    tf.gfile.MakeDirs(FLAGS.log_dir)
    train()

In [8]:
#argparse是一个受optparse启发的命令行解析库，功能：
# 1. 处理可选参数和位置参数
# 2. 生成 usage messages
# 3. 支持派生出sub-parser
# argparse包含以下public classes
# 1. ArgumentParser
# 命令行解析的主要入口点。
# 如下例所示，使用add_argument()方法为解析器填充可选参数和位置参数的动作。
# 然后调用parse_args()方法将命令行中的参数转换为具有属性的对象。
if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  parser.add_argument('--fake_data', nargs='?', const=True, type=bool,
                      default=False,
                      help='If true, uses fake data for unit testing.')
  parser.add_argument('--max_steps', type=int, default=1000,
                      help='Number of steps to run trainer.')
  parser.add_argument('--learning_rate', type=float, default=0.001,
                      help='Initial learning rate')
  parser.add_argument('--dropout', type=float, default=0.9,
                      help='Keep probability for training dropout.')
  parser.add_argument(
      '--data_dir',
      type=str,
      default=os.path.join(os.getenv('TEST_TMPDIR', '/tmp'),
                           'tensorflow/mnist/input_data'),
      help='Directory for storing input data')
  parser.add_argument(
      '--log_dir',
      type=str,
      default=os.path.join(os.getenv('TEST_TMPDIR', '/tmp'),
                           'tensorflow/mnist/logs/mnist_with_summaries1'),
      help='Summaries log directory')
  FLAGS, unparsed = parser.parse_known_args()
  tf.app.run(main=main, argv= sys.argv)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp\tensorflow/mnist/input_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp\tensorflow/mnist/input_data\train-labels-idx1-ubyte.gz
Extracting /tmp\tensorflow/mnist/input_data\t10k-images-idx3-ubyte.gz
Extracting /tmp\tensorflow/mnist/input_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Accuracy at step 0: 0.1069
Accuracy at step 10: 0.7398
Accuracy at step 20: 0.8458
Accuracy at step 30: 0.8717
Accuracy at step 40: 0.8831
Accuracy at step 50: 0.8929
Accuracy at step 60: 0.9054
Accuracy at step 70: 0.91
Accuracy at step 80: 0.9086
Accuracy at step 90: 0.91

SystemExit: 

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
